Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28, 1) (200000, 10)
Validation set (10000, 28, 28, 1) (10000, 10)
Test set (10000, 28, 28, 1) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

* DNN->CNN，将 ReLu(wx+b) 变成 ReLu(conv2d(x) +b)
* 这里模型的结构是两层CNN + 一层 NN + 线性输出层 + softmax

In [6]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels)) # 默认输入数据格式为：batch, 图形高度，图形宽度，通道数量
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(   # 定义第一层的卷积核
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(  # 第二层卷积核
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal( # shape: (7*7*depth, 64) ，全连接层
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model. 每一层由 ReLu(WX+b) 变成了 ReLu(conv2d(X)+b)
  # 模型结构为：两层 stride = 2 的 CNN -> NN ->线性输出层
  def model(data):
     #conv2d 数据默认格式为"NHWC", 数据按这样的顺序存储：[batch, in_height, in_width, in_channels]
     # strides = [1,2,2,1], 在input = data上采样时在height和width上采样的步长都为2.
     # output shape: [batch, out_height, out_width, output_channel] = [16, 14, 14, 16]
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    # 第二层
    # input: hidden shape (16, 14, 14, 16) ; output shape: (16, 7, 7, 16)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list() # [16, 7, 7, 16], 经过两个 stride = 2 的 CNN，图像尺寸变成 image_size // 4 * image_size // 4 
    # shape[0] = batch, 对应一个batch中的样本数量，因为第三层是全连接层，每个样本的数据要由三维降成一维即：shape[1] * shape[2] * shape[3]
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases) # 这一层是一个NN， ReLu(wx+b)
    return tf.matmul(hidden, layer4_weights) + layer4_biases #输出层 y = Wx + b
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [7]:

num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.354470
Minibatch accuracy: 18.8%
Validation accuracy: 17.5%
Minibatch loss at step 50: 1.247944
Minibatch accuracy: 62.5%
Validation accuracy: 56.8%
Minibatch loss at step 100: 0.703120
Minibatch accuracy: 75.0%
Validation accuracy: 71.1%
Minibatch loss at step 150: 1.022328
Minibatch accuracy: 81.2%
Validation accuracy: 74.2%
Minibatch loss at step 200: 1.043635
Minibatch accuracy: 68.8%
Validation accuracy: 77.2%
Minibatch loss at step 250: 0.884619
Minibatch accuracy: 68.8%
Validation accuracy: 74.9%
Minibatch loss at step 300: 0.280828
Minibatch accuracy: 93.8%
Validation accuracy: 78.9%
Minibatch loss at step 350: 1.006484
Minibatch accuracy: 68.8%
Validation accuracy: 80.7%
Minibatch loss at step 400: 0.402319
Minibatch accuracy: 87.5%
Validation accuracy: 80.3%
Minibatch loss at step 450: 0.321913
Minibatch accuracy: 87.5%
Validation accuracy: 81.2%
Minibatch loss at step 500: 0.299882
Minibatch accuracy: 87.5%
Validation accuracy: 80.8%
M

---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [5]:
batch_size = 16
patch_size = 5
depth = 64   #output channel
num_hidden = 64



graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels)) # 默认输入数据格式为：batch, 图形高度，图形宽度，通道数量
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(   # 定义第一层的卷积核, shape= (5,5,1,16)
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth])) 
  layer2_weights = tf.Variable(tf.truncated_normal(  # 第二层卷积核
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal( # shape: (7*7*depth, 64) ，全连接层
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  def maxpool2d(data, k = 2, s = 2, padding = 'SAME'):
      """
      kernel size: k, stride: s
      """
      return tf.nn.max_pool(data,[1,k,k,1], [1,s, s, 1], padding)
      
  
  # Model. 每一层由 WX+b -> activation 变成了 conv2d(X)+b -> activation
  def model(data):
     # output: feature map 
     # shape: [batch, out_height, out_width, output_channel] = [16, 28, 28, 16]
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    # maxpool shape: (16, 14, 14, 16)
    conv = maxpool2d(conv)
    hidden = tf.nn.relu(conv + layer1_biases)
    # 第二层
    # input: hidden shape (16, 14, 14, 16) ; output shape: (16, 14, 14, 16)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    # maxpool shape: (16,7, 7, 16)
    conv = maxpool2d(conv)
    hidden = tf.nn.relu(conv + layer2_biases)
    
    shape = hidden.get_shape().as_list() # [16, 7, 7, 16], 经过两个 stride = 2 的 CNN，图像尺寸变成 image_size // 4 * image_size // 4 
    # shape[0] = batch, 对应一个batch中的样本数量，因为第三层是全连接层，每个样本的数据要由三维降成一维即：shape[1] * shape[2] * shape[3]
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]]) # shape(16, 784)
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases) # shape (16, 64)
    return tf.matmul(hidden, layer4_weights) + layer4_biases #输出层 y = Wx + b (16, 10)
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.179543
Minibatch accuracy: 6.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 1.782326
Minibatch accuracy: 37.5%
Validation accuracy: 30.2%
Minibatch loss at step 100: 1.515747
Minibatch accuracy: 37.5%
Validation accuracy: 55.4%
Minibatch loss at step 150: 0.655346
Minibatch accuracy: 87.5%
Validation accuracy: 73.7%
Minibatch loss at step 200: 0.894538
Minibatch accuracy: 62.5%
Validation accuracy: 74.9%
Minibatch loss at step 250: 1.312760
Minibatch accuracy: 62.5%
Validation accuracy: 73.6%
Minibatch loss at step 300: 0.477994
Minibatch accuracy: 81.2%
Validation accuracy: 79.3%
Minibatch loss at step 350: 0.551404
Minibatch accuracy: 93.8%
Validation accuracy: 79.4%
Minibatch loss at step 400: 0.233533
Minibatch accuracy: 100.0%
Validation accuracy: 80.4%
Minibatch loss at step 450: 1.052486
Minibatch accuracy: 81.2%
Validation accuracy: 79.6%
Minibatch loss at step 500: 0.550707
Minibatch accuracy: 87.5%
Validation accuracy: 82.2%
M

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [7]:
batch_size = 16
patch_size = 5
depth = 16   #output channel
num_hidden = 64
beta = 0.0007


graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels)) # 默认输入数据格式为：batch, 图形高度，图形宽度，通道数量
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(   # 定义第一层的卷积核, shape= (5,5,1,16)
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth])) 
  layer2_weights = tf.Variable(tf.truncated_normal(  # 第二层卷积核
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal( # shape: (7*7*depth, 64) ，全连接层
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  

  
  # Model. 每一层由 WX+b -> activation 变成了 conv2d(X)+b -> activation
  keep_prob = tf.placeholder(tf.float32)
  def model(data, dropout = False, keep_prob = keep_prob):
     # output: feature map 
     # shape: [batch, out_height, out_width, output_channel] = [16, 28, 28, 16]
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    # maxpool shape: (16, 14, 14, 16)
    conv = maxpool2d(conv)
    hidden = tf.nn.relu(conv + layer1_biases)
    # 第二层
    # input: hidden shape (16, 14, 14, 16) ; output shape: (16, 14, 14, 16)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    # maxpool shape: (16,7, 7, 16)
    conv = maxpool2d(conv)
    hidden = tf.nn.relu(conv + layer2_biases)
    
    # dropout
    if dropout:
        hidden = tf.nn.dropout(hidden, keep_prob)
    
    shape = hidden.get_shape().as_list() # [16, 7, 7, 16], 经过两个 stride = 2 的 CNN，图像尺寸变成 image_size // 4 * image_size // 4 
    # shape[0] = batch, 对应一个batch中的样本数量，因为第三层是全连接层，每个样本的数据要由三维降成一维即：shape[1] * shape[2] * shape[3]
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]]) # shape(16, 784)
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases) # shape (16, 64)
        
    return tf.matmul(hidden, layer4_weights) + layer4_biases #输出层 y = Wx + b (16, 10)
  
  # Training computation.
  logits = model(tf_train_dataset, dropout = True)
  logits_predict = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
  l2_loss = tf.nn.l2_loss(layer1_weights) + tf.nn.l2_loss(layer2_weights) + tf.nn.l2_loss(layer3_weights) + tf.nn.l2_loss(layer4_weights)
  loss = loss + beta*l2_loss 
    
    
  # Optimizer.
  learning_rate = 0.05
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step, 100, 0.92, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_predict)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset, dropout = False))
  test_prediction = tf.nn.softmax(model(tf_test_dataset, dropout = False))

  


In [10]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))  

Initialized
Minibatch loss at step 0: 4.087697
Minibatch accuracy: 31.2%
Validation accuracy: 10.0%
Minibatch loss at step 50: 2.438468
Minibatch accuracy: 25.0%
Validation accuracy: 10.1%
Minibatch loss at step 100: 2.030589
Minibatch accuracy: 43.8%
Validation accuracy: 41.9%
Minibatch loss at step 150: 1.244756
Minibatch accuracy: 75.0%
Validation accuracy: 62.1%
Minibatch loss at step 200: 1.659234
Minibatch accuracy: 68.8%
Validation accuracy: 75.6%
Minibatch loss at step 250: 1.017142
Minibatch accuracy: 68.8%
Validation accuracy: 77.4%
Minibatch loss at step 300: 0.622330
Minibatch accuracy: 93.8%
Validation accuracy: 78.0%
Minibatch loss at step 350: 1.156829
Minibatch accuracy: 68.8%
Validation accuracy: 78.5%
Minibatch loss at step 400: 0.884395
Minibatch accuracy: 81.2%
Validation accuracy: 80.3%
Minibatch loss at step 450: 0.504304
Minibatch accuracy: 87.5%
Validation accuracy: 81.1%
Minibatch loss at step 500: 0.729079
Minibatch accuracy: 93.8%
Validation accuracy: 81.3%
M